In [ ]:
from segmentandoDatasets import segmentando_datasets
segmentando_datasets(1000,1000,1000)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint



In [ ]:
from preprocessamento import *

csv_file = 'Datasets_csv/df_PUC.csv'
train, teste, val, _, _, _ = preprocessamento(csv_file, 0.6, 0.2, 0.2, True)

In [ ]:
print(train.dtype)

print(train)

In [ ]:
"""x_train = next(train)

diff = x_train[0][0] - x_train[1][0]
print(np.sum(diff))"""

In [ ]:
images, labels = next(train)

fig, axes = plt.subplots(1, 16, figsize=(15, 5))

for i in range(16):
    axes[i].imshow(np.clip(images[i], 0, 255).astype('uint8'))
    axes[i].axis('off') 

plt.show()

In [ ]:
class NormalizedDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data_iterator):
        self.data_iterator = data_iterator

    def __len__(self):
        return len(self.data_iterator)

    def __getitem__(self, idx):
        batch = self.data_iterator[idx]
        if isinstance(batch, tuple):
            return (batch[0] / 255.0, batch[0] / 255.0)  # Normalize input and target
        else:
            return batch / 255.0

In [ ]:
class Autoencoder(Model):
    def __init__(self, input_shape=(256, 256, 3)):
        super(Autoencoder, self).__init__()
        self.input_shape = input_shape

        self.encoder = tf.keras.Sequential([
            layers.Input(shape=input_shape),
            layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2),
            layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=2),
            layers.Conv2D(128, (3, 3), activation='relu', padding='same', strides=2),
            layers.Conv2D(256, (3, 3), activation='relu', padding='same', strides=2)
        ])

        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(128, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv2DTranspose(64, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv2DTranspose(32, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv2DTranspose(3, kernel_size=3, strides=2, activation='sigmoid', padding='same')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
"""def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    return lr"""

In [ ]:
autoencoder = Autoencoder()

In [ ]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
"""autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                    loss='mse')"""

In [ ]:
from tensorflow.keras import layers, Model, callbacks


checkpoint_path = 'weights_parciais/weights-improvement-{epoch:02d}-{loss:.2f}.weights.h5'
cp_callback = ModelCheckpoint(filepath=checkpoint_path, 
                                  save_weights_only=True, 
                                  monitor='loss',
                                  mode='max', 
                                  save_best_only=True, 
                                  verbose=1)

"""cp_callback = callbacks.ModelCheckpoint(filepath='best_model.keras',
                                        save_best_only=True,
                                        monitor='val_loss')
early_stopping = callbacks.EarlyStopping(monitor='val_loss',
                                         patience=10,
                                         restore_best_weights=True)
lr_scheduler = callbacks.LearningRateScheduler(lr_schedule)"""



In [ ]:
"""train_gen = NormalizedDataGenerator(train)
val_gen = NormalizedDataGenerator(val)"""

In [ ]:
history = autoencoder.fit(train,
                          epochs=50,
                          validation_data=val,
                          callbacks=[cp_callback],
                          verbose=1)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.title('Training Loss (Log Scale)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.yscale('log')
plt.show()

In [ ]:
autoencoder.save('Modelos_keras/Autoencoder2.keras')
autoencoder.save_weights('weights_finais/Autoencoder2.weights.h5')

In [ ]:
autoencoder.load_weights('weights_finais/Autoencoder2.weights.h5')

In [ ]:
encoded_imgs = autoencoder.encoder(train[0])
decoded_imgs = autoencoder.decoder(encoded_imgs)

In [ ]:
print(encoded_imgs.shape)
print(decoded_imgs.shape)

In [ ]:
#x_test = next(teste)

In [ ]:
print(np.unique(decoded_imgs[0]))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

input_imgs, _ = train[0]  # Extraia as imagens

num_imgs = min(encoded_imgs.shape[0], 5)  # Por exemplo, visualize no máximo 5 imagens
num_channels = encoded_imgs.shape[-1]  # Número de canais
num_channels_to_show = min(num_channels, 10)  # Número de canais a mostrar por imagem

fig, axes = plt.subplots(3, num_imgs, figsize=(15, 9))

for i in range(num_imgs):
    axes[0, i].imshow(np.clip(input_imgs[i], 0, 255).astype('uint8'))
    axes[0, i].axis('off')
    axes[0, i].set_title('Original')

for i in range(num_imgs):
    axes[1, i].imshow(np.clip(decoded_imgs[i], 0, 255).astype('uint8'))
    axes[1, i].axis('off')
    axes[1, i].set_title('Decodificado')

for i in range(num_imgs):
    encoded_img = encoded_imgs[i]
    for j in range(num_channels_to_show):
        axes[2, i].imshow(encoded_img[:, :, j], cmap='gray')
        axes[2, i].axis('off')
        axes[2, i].set_title(f'Codificada')

plt.tight_layout()
plt.show()


In [ ]:
#UFPR04, _ = preprocessamento_dataframe_unico("Datasets_csv/df_UFPR04.csv", True)

In [ ]:
"""x_UFPR04, _ = next(UFPR04)

x_UFPR04 = x_UFPR04.astype('float32')/255.

print(x_UFPR04.shape)"""